# RAG Experiment

This notebook runs two experiments:
1. Query the LLM directly for a set of questions and store the answers.
2. Load PDF documents into Qdrant and re-run the questions using retrieval augmented generation (RAG).


In [1]:
import json
from pathlib import Path

from qdrant_utils import (
    answer_with_context,
    get_qdrant_client,
    load_pdf_and_chunk,
    embed_chunks,
    store_embeddings_in_qdrant,
    retrieve_similar_chunks,
    evaluate_llm_as_judge,
)

from experiment_utils import (
    parse_judge_response,
    load_processed_ids,
    append_result_to_csv,
    precision_n,
    recall_n,
    f1_n
)

DATA_PATH = 'question.json'
PDF_DIR = Path('papers')
BASELINE_CSV = "results_baseline.csv"
RAG_CSV = 'results_rag.csv'
COLLECTION_NAME = 'rag_papers_final2'


/Users/I545843/Documents/DHBW/4. Semester/Datenbanken/Datenbanken/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Load evaluation questions
The JSON file contains questions grouped by paper and additional metadata questions.
We flatten them into a single list with their gold answers.

In [2]:
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)['evaluation_dataset']

questions = []
# content questions per paper
for paper in data['papers']:
    for q in paper.get('questions', []):
        questions.append({
            'question_id': q['question_id'],
            'question': q['question'],
            'answer': q['answer']
        })

len(questions)


35

## Experiment 1: direct LLM answers
Each question is sent to the LLM without any additional context. The answers are saved to *results_ground_truth.csv*.

In [3]:
processed_ids = load_processed_ids(BASELINE_CSV)

for q in questions:
    qid = q["question_id"]
    if qid in processed_ids:
        continue

    llm_answer = answer_with_context(q["question"], [])
    gold = q["answer"]

    # ROUGE-1
    p1 = precision_n(llm_answer, gold, 1)
    r1 = recall_n(llm_answer, gold, 1)
    f1_1 = f1_n(p1, r1)

    # ROUGE-2
    p2 = precision_n(llm_answer, gold, 2)
    r2 = recall_n(llm_answer, gold, 2)
    f1_2 = f1_n(p2, r2)

    # LLM-as-a-Judge
    raw_judge = evaluate_llm_as_judge(
        question_id=qid,
        question=q["question"],
        answer_llm=llm_answer,
        answer_gold=gold,
        model="gpt-4o"
    )
    judge_scores = parse_judge_response(raw_judge)

    row = {
        "question_id": qid,
        "question_string": q["question"],
        "answer_llm": llm_answer,
        "answer_gold": gold,
        "precision-1": p1,
        "recall-1": r1,
        "ROUGE-1": f1_1,
        "precision-2": p2,
        "recall-2": r2,
        "ROUGE-2": f1_2,
        "factual_correctness": judge_scores["factual_correctness"],
        "completeness":        judge_scores["completeness"],
        "relevance":           judge_scores["relevance"],
        "justification":       judge_scores["justification"],
        "depth":               judge_scores["depth"],
        "overall_score":       judge_scores.get("overall_score"),
        "pass":                judge_scores.get("pass")
    }

    append_result_to_csv(row, BASELINE_CSV)
    processed_ids.add(qid) 


## Load PDF documents into Qdrant
All PDF files are chunked, embedded and stored in the collection defined above.

In [ ]:
client = get_qdrant_client()
for pdf in PDF_DIR.glob('*.pdf'):
    chunks = load_pdf_and_chunk(str(pdf))
    embeddings = embed_chunks(chunks)
    store_embeddings_in_qdrant(client, COLLECTION_NAME, chunks, embeddings)


## Experiment 2: RAG answers
For each question we retrieve relevant chunks from Qdrant and pass them to the LLM. Results are written to *results_rag.csv*.

In [ ]:
processed_ids = load_processed_ids(RAG_CSV)

for q in questions:
    qid = q["question_id"]
    if qid in processed_ids:
        continue

    context = retrieve_similar_chunks(q["question"], client, COLLECTION_NAME, top_k=5)
    llm_answer = answer_with_context(q["question"], context)
    gold = q["answer"]

    # ROUGE-1
    p1 = precision_n(llm_answer, gold, 1)
    r1 = recall_n(llm_answer, gold, 1)
    f1_1 = f1_n(p1, r1)

    # ROUGE-2
    p2 = precision_n(llm_answer, gold, 2)
    r2 = recall_n(llm_answer, gold, 2)
    f1_2 = f1_n(p2, r2)

    # LLM-as-a-Judge
    raw_judge = evaluate_llm_as_judge(
        question_id=qid,
        question=q["question"],
        answer_llm=llm_answer,
        answer_gold=gold,
        model="gpt-4o"
    )
    judge_scores = parse_judge_response(raw_judge)

    row = {
        "question_id": qid,
        "question_string": q["question"],
        "answer_llm": llm_answer,
        "answer_gold": gold,
        "precision-1": p1,
        "recall-1": r1,
        "ROUGE-1": f1_1,
        "precision-2": p2,
        "recall-2": r2,
        "ROUGE-2": f1_2,
        "factual_correctness": judge_scores["factual_correctness"],
        "completeness":        judge_scores["completeness"],
        "relevance":           judge_scores["relevance"],
        "justification":       judge_scores["justification"],
        "depth":               judge_scores["depth"],
        "overall_score":       judge_scores.get("overall_score"),
        "pass":                judge_scores.get("pass")
    }

    append_result_to_csv(row, RAG_CSV)
    processed_ids.add(qid)  # avoid re-processing
